# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
* Usage of Strings
* Method Signatures (Modifiers, Parameter Types, Number of Parameters...)
* Other methods in same class
* Analyze Method Calls (from and to) via Call Graph (Distance, Offsets, Graph Analysis)

In [ ]:
%matplotlib notebook


from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [ ]:
import unittest
from collections import defaultdict, Counter
from os import path

from androguard.core.analysis.analysis import MethodAnalysis, ClassAnalysis, FieldAnalysis
from androguard.core.bytecode import FormatClassToJava
from androguard.misc import AnalyzeAPK
from androguard.session import Save, Session, Load

from formats import *
from decs import *

from matching import matcher, strings, structures, methods

from start import process_files

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [ ]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues


# Matching Rules
strings.MAX_USAGE_COUNT_STR = 20
strings.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods.MIN_MATCH_POINTS = 2



# APK Files to load
file_paths = (
    "../../../Downloads/com.snapchat.android_10.85.5.74-2067_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.snapchat.android_10.86.5.61-2069_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)

In [ ]:
(a, d, dx), (a2, d2, dx2) = process_files(file_paths, MULTIPROCESS_FILES)

### Utility Functions to work with Androguard and Java Representations

* Converting Parameter types to TypeDescriptor Format
* Strip return type (not used for hooking)
* Method Representation Format

Loaded with Unit Tests

# Method Declarations

Lightweight Method Declaration for internal representation of a Method / Hook.

Not keeping Androguard Objects in memory to avoid high memory usage.

### List of Methods

Defining the list of methods to find (obviously requires full class names)

In [ ]:
decs_to_find = (
    MethodDec("rD5", "a", "rD5", "qD5"),
    MethodDec("MSg", "j0", "SGd"),
    MethodDec("x45", "h"),
    MethodDec("GIb", "<init>", skip_params=True)
)

# Processing

## Strings as Characteristics

Extracting Strings used either in the given methods directly or in the classes the methods define

In [ ]:
resolved_classes = resolve_classes(dx, decs_to_find)
resolved_methods = resolve_methods(decs_to_find, resolved_classes)
decs_ma = dict(zip(decs_to_find, resolved_methods))

print("Resolved all Classes and Methods")
if False:
    print("", *map(pretty_format_ma,resolved_methods), sep="\n* ")

### Utility functions for working with dx.get_strings()

Filters Strings and xrefs to Strings. Only allow strings with (#xrefs < MAX_USAGE_COUNT_STR) to be used as characteristic to locate classes

Building Maps of MethodDec and ClassNames associated to lists containing strings used in them

### Count occurrences of strings

Converting list of strings to a Counter object for faster comparisons

### Searching for Found Strings

Tries to resolve Classes and methods with the strings previously found

* Loading second Apk File
* Find All Strings found previously, build Map of potential matches (ClassName/Method to Counter)
* Filter Potential Matches by comparing both Counter Objects

#### Try to resolve Method
Try to resolve classes by only using information about Strings (exact Counter Match)

In [ ]:
accumulator = matcher.Accumulator()

args = (dx, dx2, resolved_classes, decs_ma)

In [ ]:
string_matcher = strings.StringMatcher(*args, accumulator.get_unmatched_ms(decs_to_find))
candidates_cs, candidates_ms = string_matcher.compare_counters()

accumulator.add_candidates(candidates_cs, candidates_ms)

### Fallback by unique strings

To resolve unresolved methods, get all unique strings (strings only used by this class) and try to find the matching class by only searching for the unique string.

In [ ]:
candidates_cs = string_matcher.compare_unique_strings(accumulator.get_unmatched_cs(decs_to_find))

print()
print("| Applying Filtered Candidates to Accumulator")
print()
accumulator.add_candidates(candidates_cs)

### Using Class Information

Gathers the following information and tries to find the correct classes by finding a similar "Profile"
* Modifiers for Methods and Fields
* "static" Field and return types of Methods
* #Fields and #Methods

In [ ]:
structure_matcher = structures.StructureMatcher(*args, accumulator.get_unmatched_ms(decs_to_find))
candidates_cs = structure_matcher.get_exact_structure_matches(accumulator.get_unmatched_cs(decs_to_find))

print("| Applying Candidates to Accumulator")
print()
accumulator.add_candidates(candidates_cs)

### Fallback if Class was found

In case the class was found, but the method could not be resolved, check each method of the class for the following criteria:

* Matching #xrefs_to
* Matching #xrefs_from
* Matching Code length

All of these checks are currently strict/exact

In [ ]:
import importlib
importlib.reload(methods)

method_matcher = methods.MethodMatcher(*args, accumulator.get_unmatched_ms(decs_to_find))

candidates_ms = method_matcher.try_resolve_ms(accumulator.get_unmatched_cs(decs_to_find), accumulator.matching_cs, True)
accumulator.add_candidates(candidates_ms=candidates_ms)


candidates_ms = method_matcher.try_resolve_ms(accumulator.get_unmatched_cs(decs_to_find), accumulator.matching_cs, False)
accumulator.add_candidates(candidates_ms=candidates_ms)